# 示例解析
## 账本应用
程序在进入时，会创建TradingEnv类的实例env：
```python
env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.SimpleOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)
```
这里我们先来研究action_scheme属性。重点关注evn.action_scheme.action_space属性。由上面的代码可以看出，env.action_scheme是iqt.env.default.actions.SimpleOrders类，action_space是其计算属性，如下所示：
```python
@property
def action_space(self) -> Space:
    if not self._action_space:
        self.actions = product(
            self.criteria,
            self.trade_sizes,
            self.durations,
            [TradeSide.BUY, TradeSide.SELL]
        )
        print('self.actions 1: {0};'.format(self.actions))
        self.actions = list(self.actions)
        print('self.actions 2: {0};'.format(self.actions))
        self.actions = list(product(self.portfolio.exchange_pairs, self.actions))
        print('self.actions 3: {0};'.format(self.actions))
        self.actions = [None] + self.actions
        print('self.actions 4: {0};'.format(self.actions))
        self._action_space = Discrete(len(self.actions))
    return self._action_space
```
运行结果如下所示：
```bash
self.actions 1: <itertools.product object at 0x000001B16C0F53C0>;
self.actions 2: 
[
    (None, 0.1, None, <TradeSide.BUY: 'buy'>), 
    (None, 0.1, None, <TradeSide.SELL: 'sell'>), 
    (None, 0.2, None, <TradeSide.BUY: 'buy'>), 
    ......
    (None, 0.9, None, <TradeSide.SELL: 'sell'>), 
    (None, 1.0, None, <TradeSide.BUY: 'buy'>), 
    (None, 1.0, None, <TradeSide.SELL: 'sell'>)
];

self.actions 3: 
[
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
    
self.actions 4: 
[
    None, 
    ################################################################################################
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.SELL: 'sell'>)),
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
action_space: Discrete(101);
```
其中action_space为gym.spaces.Descrete(101)，其取值范围为0$\sim$100，对应于step4打印出的某一行，第一项为操作标的，第二项为：
1. cretia：代表市价、限价等订单类型；
2. trade_size：操作比例，如bitstamp: USD/ETH, (None, 0.2, None, <TradeSide.BUY: 'buy'>代表用当前资金的20%买入ETH；
3. duration：None；
4. 买卖方向；

当为1$\sim$100时，代表对交易对的买卖操作。当为0时，代表什么都不做。

接下来对环境进行重复，然后进入无限循环，在此无限循环的每一步，首先从action_space中随机取出一个操作，然后调用env.step方法：

```python
test
```
$$
E = m \times C^{2}
$$